In [1]:
%load_ext autotime
import sys
# sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/magellan/')
sys.path.append('/scratch/pradap/python-work/anhaidgroup/py_entitymatching/')

This quickstart guide explains how to match two tables using Magellan. Our goal is to come up with a workflow to match DBLP and ACM datasets. Specifically, we want to achieve precision greater than 95% and get recall as high as possible. The datasets contain information about the conference papers published in top databse conferences. 

First, we need to import the Magellan package as follows:

In [2]:
# Import libraries
import py_entitymatching as em
import pandas as pd
import os, sys

time: 1.18 s


In [3]:
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.5.2 |Anaconda 4.1.1 (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
pandas version: 0.18.1
magellan version: 0.1.0
time: 1.36 ms


Matching two tables typically consists of the following three steps:
1. Loading the input tables
2. Blocking the input tables to get a candidate set
3. Matching the tuple pairs in the candidate set

# 1. Loading the input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [4]:
dblp_dataset_path = os.sep.join(['DBLP_ACM', 'DBLP_cleaned.csv'])
acm_dataset_path = os.sep.join(['DBLP_ACM', 'ACM_cleaned.csv'])

time: 76.8 ms


In [5]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(dblp_dataset_path, key='id')
B = em.read_csv_metadata(acm_dataset_path, key='id')

time: 130 ms


In [6]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2616
Number of tuples in B: 2294
Number of tuples in A X B (i.e the cartesian product): 6001104
time: 63.6 ms


In [7]:
A.head(2)

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996


time: 115 ms


In [8]:
B.head(2)

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999


time: 65.7 ms


In [9]:
# Display the key attributes of table A and B.
em.get_key(A), em.get_key(B)

('id', 'id')

time: 74.5 ms


In [10]:
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 500, 1)
# But for the demo, we will use the entire table A and B

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


time: 934 ms


# 2. Blocking to create candidate tuple pairs

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching. 

Magellan provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. Refer to [api reference] for more details. The user can mix and match these blockers to form a blocking sequence 
applied to input tables.

For the matching problem at hand, we know that two conference papers published in different years cannot match. So, we decide to apply an attribute equivelance blocker on the 'year' attribute. 

In [11]:
# Plan
# A, B ------ attribute equivalence [year] -----> C1

time: 579 µs


In [12]:
# Create attribute equivalence blocker
ab = em.AttrEquivalenceBlocker()
# Block tables using 'year' attribute: same year then include in the canidate set
C1 = ab.block_tables(A, B, 'year', 'year', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

time: 231 ms


In [13]:
# Check the number of rows in C1
len(C1)

601284

time: 2.24 ms


In [14]:
# Display first two rows from C1
C1.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/sigmod/Mackay99,304586,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",1999
1,1,journals/sigmod/Mackay99,304587,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",1999


time: 169 ms


The number of tuple pairs considered for matching is reduced to 601284 (from 6001104), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in Magellan as follows:

In [15]:
# Debug blocker output
dbg = em.debug_blocker(C1, A, B, output_size=200)

time: 1.93 s


In [16]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head()

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.875,journals/sigmod/Aberer02,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
1,1,0.875,journals/sigmod/Aberer02a,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
2,2,0.875,journals/sigmod/Aberer02b,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
3,3,0.875,journals/sigmod/Aberer03b,601865,Book review column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
4,4,0.875,journals/sigmod/Aberer02,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record


time: 13.9 ms


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the conference papers to match the author names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [17]:
# Updated blocking sequence
# A, B ------ attribute equivalence [year] -----> C1--
#                                                     |----> C
# A, B ------ overlap blocker [authors] --------> C2--

time: 594 ms


In [18]:
# Create an overlap blocker
ob = em.OverlapBlocker()
# Apply overlap blocker on 'authors' attribute
C2 = ob.block_tables(A, B, 'authors', 'authors', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


time: 1.62 s


In [19]:
# Check the number of rows in C2
len(C2)

287414

time: 1.82 ms


In [20]:
# Display first two rows from C2
C2.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/tods/LechtenborgerV03,304586,On the computation of relational view complements,lechtenbrger vossen jens gottfried,2003,The WASA2 object-oriented workflow management system,vossen weske mathias gottfried,1999
1,1,conf/sigmod/VossenW99,304586,The WASA2 Object-Oriented Workflow Management System,vossen weske mathias gottfried,1999,The WASA2 object-oriented workflow management system,vossen weske mathias gottfried,1999


time: 209 ms


In [21]:
# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([C1, C2])

time: 815 ms


In [22]:
# Check the number of rows in the consolidated candidate set.
len(C)

857777

time: 1.79 ms


We observe that the number of tuple pairs considered for matching is increased to 875758 (from 601284). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [23]:
# Debug again
dbg = em.debug_blocker(C, A, B)

time: 3.78 s


In [24]:
# Display first few rows from the debugger output
dbg.head(3)

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.500000,journals/sigmod/Dogac98,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,ACM SIGMOD Record
1,1,0.500000,journals/sigmod/Dogac02,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,ACM SIGMOD Record
2,2,0.461538,journals/sigmod/Snodgrass98a,641001,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,ACM SIGMOD Record


time: 13.8 ms


We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.

# 3. Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.

This typically involves the following five steps:

1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## 3.1 Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [25]:
# Sample candidate set
S = em.sample_table(C, 450)

time: 361 ms


Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [26]:
# Label S and specify the attribute name for the label column
L = em.label_table(S, 'gold')

time: 2.73 s


For the purposes of this guide, we will load in a pre-labeled dataset (of 415 tuple pairs) included in this package.

In [27]:
# Load the pre-labeled data
L = em.read_csv_metadata('DBLP_ACM/dblp_acm_demo_labels.csv', ltable=A, rtable=B)
# Display the number of rows in the labaled data set
len(L)

415

time: 8.12 ms


## 3.2 Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development and evaluation. Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [28]:
# Split the labeled data into development and evaluation set
development_evaluation = em.split_train_test(L, train_proportion=0.7)
development =  development_evaluation['train']
evaluation = development_evaluation['test']

time: 263 ms


## 3.3 Select the best learning-based matcher

Selecting the best learning-based matcher typically involves the following steps:
1. Creating a set of learning-based matchers
2. Creating features
3. Extracting feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation
5. Debugging the matcher (and possibly repeat the above steps)

### 3.3.1 Creating a set of learning-based matchers

First, we need to create a set of learning-based matchers. The following matchers are supported in Magellan: (1) decision tree, (2) random forest, (3) naive bayes, (4) svm, (5) logistic regression, and (6) linear regression.

In [29]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree')
svm = em.SVMMatcher(name='SVM')
rf = em.RFMatcher(name='RF')
nb = em.NBMatcher(name='NB')
lg = em.LogRegMatcher(name='LogReg')
ln = em.LinRegMatcher(name='LinReg')

time: 742 ms


### 3.3.2 Creating features

Next, we need to create a set of features for the development set. Magellan provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [30]:
# Generate features
feature_table = em.get_features_for_matching(A, B)

time: 810 ms


In [31]:
# List the names of the features generated
feature_table['feature_name']

0             title_title_jac_qgm_3_qgm_3
1         title_title_cos_dlm_dc0_dlm_dc0
2                         title_title_mel
3                    title_title_lev_dist
4                     title_title_lev_sim
5         authors_authors_jac_qgm_3_qgm_3
6     authors_authors_cos_dlm_dc0_dlm_dc0
7                     authors_authors_mel
8                authors_authors_lev_dist
9                 authors_authors_lev_sim
10                          year_year_exm
11                          year_year_anm
12                     year_year_lev_dist
13                      year_year_lev_sim
Name: feature_name, dtype: object

time: 22 ms


We observe that there were 14 features generated. As a first step, lets say that we decide to use only 'year' related features.

In [32]:
# Select the year related features
feature_subset_iter1 = feature_table[10:14]

time: 61.5 ms


In [33]:
# List the names of the features selected
feature_subset_iter1['feature_name']

10         year_year_exm
11         year_year_anm
12    year_year_lev_dist
13     year_year_lev_sim
Name: feature_name, dtype: object

time: 73 ms


### 3.3.3 Extracting feature vectors

In this step, we extract feature vectors using the development set and the created features.

In [34]:
# Extract feature vectors
feature_vectors_dev = em.extract_feature_vecs(development, 
                            feature_table=feature_subset_iter1, 
                            attrs_after='gold')                                         

0%                          100%
[##############################] | ETA: 00:00:00

time: 317 ms



Total time elapsed: 00:00:00


In [35]:
# Display first few rows
feature_vectors_dev.head(3)

,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold
171,171,conf/vldb/DonjerkovicR99,671510,1,1.0,0.0,1.0,1
301,301,journals/sigmod/Hanna95,212001,1,1.0,0.0,1.0,0
91,91,conf/sigmod/NguyenACP01,375723,1,1.0,0.0,1.0,1


time: 10.7 ms


Next, we might have to impute the feature vectors as it might contain missing values. First, let us check if there are any missing values in the extracted feature vectors.

In [36]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_dev))

True

time: 63.7 ms


We observe that the extracted feature vectors contain missing values. We have to impute the missing values for the learning-based matchers to fit the model correctly. For the purposes of this guide, we impute the missing value in a column with the mean of the values in that column. 

In [37]:
# Impute feature vectors with the mean of the column values.
feature_vectors_dev = em.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

time: 85.4 ms


### 3.3.4 Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' metric to select the best matcher.

In [54]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=10,
        target_attr='gold', metric='precision') 

time: 272 ms


In [55]:
# Check the cross validation statistics
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f02d94f6c88>,10,1.0,1.000000,1.0,1.000000,1.0,1.0,0.937500,1.0,1.000000,1.0,0.993750
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f02d94f6d68>,10,1.0,1.000000,1.0,1.000000,1.0,1.0,0.937500,1.0,0.950000,1.0,0.988750
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f02d94f6cc0>,10,1.0,0.923077,1.0,1.000000,1.0,1.0,1.000000,1.0,0.947368,1.0,0.987045
3,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f02d94f6dd8>,10,1.0,1.000000,1.0,0.941176,1.0,1.0,0.937500,1.0,1.000000,1.0,0.987868
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f02d94f6e80>,10,1.0,0.933333,1.0,1.000000,1.0,1.0,0.882353,1.0,0.950000,1.0,0.976569
5,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f02d94f6ef0>,10,1.0,1.000000,1.0,1.000000,1.0,1.0,0.937500,1.0,0.950000,1.0,0.988750


time: 19.6 ms


### 3.3.5 Debugging matcher

We observe that the best matcher is not getting us to the precision that we expect (i.e > 95%). We debug the matcher to see what might be wrong.

To do this, first we split the feature vectors into train and test.

In [40]:
# # Split feature vectors into train and test
train_test = em.split_train_test(feature_vectors_dev, train_proportion=0.5)
train = train_test['train']
test = train_test['test']

time: 526 ms


Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [41]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold')

time: 1.68 s


From the GUI, we observe that using only 'year' related features result in a lot of false positives. So we decide to use all the features in the feature table (which had author, title and year related features).

In [42]:
# Select all features from the feature table
feature_subset_iter2 = feature_table

time: 804 µs


Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [43]:
# Get new set of features
feature_vectors_dev = em.extract_feature_vecs(development, feature_table=feature_subset_iter2, attrs_after='gold')

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


time: 691 ms


In [44]:
# Check if imputation is required
any(pd.isnull(feature_vectors_dev))

True

time: 2.7 ms


In [45]:
# Impute feature vectors
feature_vectors_dev = em.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

time: 115 ms


In [57]:
# Apply cross validation to find if there is a better matcher
result = em.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=10,
        target_attr='gold', metric='f1') 

time: 270 ms


In [58]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f02d94f6c88>,10,0.97561,1.000000,1.0,1.000000,0.972973,1.000000,0.972973,1.000000,0.95000,0.967742,0.983930
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f02d94f6d68>,10,0.97561,1.000000,1.0,1.000000,0.972973,1.000000,0.972973,1.000000,0.97561,1.000000,0.989717
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f02d94f6cc0>,10,0.95000,0.928571,1.0,0.916667,0.944444,0.888889,0.941176,0.944444,0.95000,0.896552,0.936074
3,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f02d94f6dd8>,10,0.97561,0.967742,1.0,1.000000,0.972973,1.000000,0.972973,1.000000,1.00000,1.000000,0.988930
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f02d94f6e80>,10,0.97561,0.937500,1.0,0.962963,0.972973,1.000000,0.972973,1.000000,0.97561,0.967742,0.976537
5,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f02d94f6ef0>,10,1.00000,1.000000,1.0,1.000000,0.972973,1.000000,0.972973,1.000000,0.97561,1.000000,0.992156


time: 18.1 ms


Now, observe the best matcher is achieving the expected precision and we can proceed on to evaluating the best matcher on the unseen data (the evaluation set).

## 3.4 Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Extracting the feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### 3.4.1 Extracting the feature vectors

As before, we extract the feature vectors (using the updated feature table and the evaluation set) and impute it (if necessary).

In [48]:
# Get new set of features
feature_vectors_eval = em.extract_feature_vecs(evaluation, 
                                               feature_table=feature_subset_iter2, 
                                               attrs_after='gold')


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


time: 430 ms


In [49]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_eval))

True

time: 2.75 ms


In [50]:
# Impute feature vectors
feature_vectors_eval = em.impute_table(feature_vectors_eval, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

time: 82.2 ms


### 3.4.2 Training the matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [51]:
# Train using feature vectors from the development set
rf.fit(table=feature_vectors_dev, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')


time: 124 ms


### 3.4.3 Predicting the matches
Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [52]:
# Predict M 
predictions = rf.predict(table=feature_vectors_eval, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

time: 74.3 ms


### 3.4.4 Evaluating the matching output

Finally, we evaluate the predicted outputs

In [53]:
# Evaluate the result
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)

Precision : 98.28% (57/58)
Recall : 98.28% (57/58)
F1 : 98.28%
False positives : 1 (out of 58 positive predictions)
False negatives : 1 (out of 67 negative predictions)
time: 104 ms
